Version 1.0.0

# Introduction

In this programming assignment we will illustrate a very severe data leakage, that can often be found in competitions, where the pairs of object should be scored, e.g. predict $1$ if two objects belong to the same class and $0$ otherwise. 

The data in this assignment is taken from a real competition, and the funniest thing is that *we will not use training set at all* and achieve almost 100% accuracy score! We will just exploit the leakage.

Now go through the notebook and complete the assignment.

## Coop's Analysis of the Data Leakage
I have spent hours on this and while I was able to get the code to work I am nut sure if I really understand what is going on. I will try to summarize

We know that 50% of the test set is positive (class 1 out of [0,1])

We know that the total number of pairs is much higher than the number of positive pairs in the test set. This suggests that the test set was constructed using an algorithm. Perhaps we can exploit this or even determine what the algorithm was and use it to predict the classes of the test set.

We calculated the similarity between "what the FirstIDs connect to" and "what the SecondIds connect to" using a dot product of sparse matrices. The result is a similarity score for each pairId that describes how similar the connectivity of each Id in the pair are to each other.

The higher the similarity score, the more likely that the two Ids in the pair should be classified as a 1.

We identified a threshold of similarity score that we should use for each pairID and chose 1 when the similarity score is above this threshold. We know to choose 1 when the score is above the threshold because as the score increases, the similarity between the connectivity increases. As the similarity increases, the likelihood of belonging to the same class increases.

We determined the threshold by examining the number of occurrences of each similar score and compared with the number of 1's that we expected (184,225).  Setting f > 15 gets us closest to that value.

This achieved an accuracy score of 0.999609.

To summarize, we can get near perfect predictions because we know there is a strong relationship (i.e. correlation) between "what the FirstIDs connect to" and "what the SecondIds connect to."

In [ ]:
# pairId is a combination of a FirstId and a SecondId. It can be modeled as an edge.
# there are 368,550 such edges in the test set, but there are 346,489,650 potential pairIds.
# of these 368,550 edges, we know that 184,275 are in class 1 and 184, 275 in class 0 because 50% are in one class
# there are 26,325 unique First/Second Ids

# create an incidence matrix that has First/Second Ids for both row and column indices. 
# the dimensionality of this matrix is (# First/Second Ids + 1, # First/Second Ids + 1)
#    mark the coordinates in the matrix corresponding to the pairId (FirstId, SecondId) 
#    and its reverse (SecondId, FirstId) with a 1. the rest are zeros. use sparse matrix

# in the incidence matrix, there are 736,872 ones and many, many more zeros. 

# the incidence matrix thus represents the connectivity between FirstIds and SecondIs

# we will then measure the similiarty of what FirstIds connect to with what SecondIds connect to. 
# we do this by taking the dot produce of 'FirstId connectivity' with 'SeconfId connectivity'
# 'FirstId connectivity' can be derived from the incicence matrix by taking only the rows with
#   row indces that correspond to FirstIds. 
# the same is true for 'SecondId connctivity'

# the dot product represents the degree of similiarity between 'FirstId connectivity' and 'SecondId connectivity'

# there are only a few unique scores for the dot product and we know how often those scores occur. 
# we can use the score as a feature because each FirstId / SecondId pair will have a score. 

# since we know that 50% of the test data should be marked as 1, we know the exact number (184,225). 
# we can use the score to determine whether to mark a class as a 0 or 1. 
# the closest we can get to 184,225 is to set all Pairs to 1 if the score (f) is greater than 14

In [2]:
import numpy as np
import pandas as pd 
import scipy.sparse

# Load the data

Let's load the test data. Note, that we don't have any training data here, just test data. Moreover, *we will not even use any features* of test objects. All we need to solve this task is the file with the indices for the pairs, that we need to compare.

Let's load the data with test indices.

In [3]:
test = pd.read_csv('../readonly/data_leakages_data/test_pairs.csv')
test.head(10)

,pairId,FirstId,SecondId
0,0,1427,8053
1,1,17044,7681
2,2,19237,20966
3,3,8005,20765
4,4,16837,599
5,5,3657,12504
6,6,2836,7582
7,7,6136,6111
8,8,23295,9817
9,9,6621,7672


For example, we can think that there is a test dataset of images, and each image is assigned a unique `Id` from $0$ to $N-1$ (N -- is the number of images). In the dataframe from above `FirstId` and `SecondId` point to these `Id`'s and define pairs, that we should compare: e.g. do both images in the pair belong to the same class or not. So, for example for the first row: if images with `Id=1427` and `Id=8053` belong to the same class, we should predict $1$, and $0$ otherwise. 

But in our case we don't really care about the images, and how exactly we compare the images (as long as comparator is binary).  

**We suggest you to try to solve the puzzle yourself first.** You need to submit a `.csv` file with columns `pairId` and `Prediction` to the grader. The number of submissions allowed is made pretty huge to let you explore the data without worries. The returned score should be very close to $1$.

**If you do not want to think much** -- scroll down and follow the instructions below.

In [4]:
test.describe()

,pairId,FirstId,SecondId
count,368550.000000,368550.000000,368550.000000
mean,184274.500000,10863.601118,11950.398882
std,106391.365192,7280.190939,7602.814820
min,0.000000,0.000000,0.000000
25%,92137.250000,4574.000000,5698.000000
50%,184274.500000,9886.000000,10512.000000
75%,276411.750000,16512.000000,18782.000000
max,368549.000000,26324.000000,26324.000000


In [37]:
test[test['FirstId'] == test['SecondId']]
# there are 6 pairIds where FirstId and SecondId are the same

,pairId,FirstId,SecondId
186443,186443,300,300
218927,218927,2551,2551
221189,221189,8046,8046
228783,228783,2799,2799
293834,293834,292,292
327330,327330,10404,10404


In [6]:
print(test['FirstId'].nunique())
print(test['SecondId'].nunique())

26325
26310


In [55]:
print(test[test.duplicated(subset=['FirstId', 'SecondId'])].shape)
print(test[test.duplicated(subset=['FirstId', 'SecondId'], keep = False)].shape)

# there are 12 pairs of FirstId, Second Id that are repeated

(12, 3)
(24, 3)


# EDA and leakage intuition

As we already know, the key to discover data leakages is careful EDA. So let's start our work with some basic data exploration and build an intuition about the leakage.

First, check, how many different `id`s are there: concatenate `FirstId` and `SecondId` and print the number of unique elements. Also print minimum and maximum value for that vector.

In [22]:
# YOUR CODE GOES HERE
concat = test['FirstId'].append(test['SecondId'])
print(concat.nunique(), concat.unique().min(), concat.unique().max())

# there are 26,325 unique First/Second Ids

26325 0 26324


and then print how many pairs we need to classify (it is basically the number of rows in the test set)

In [9]:
# YOUR CODE GOES HERE
test.shape[0]

368550

Now print, how many distinct pairs it would be possible to create out of all "images" in the dataset?   

In [10]:
# YOUR CODE GOES HERE
distinct_pairs = (26325)*(26325-1)/2 
distinct_pairs

346489650.0

So the number of pairs we are given to classify is very very small compared to the total number of pairs. 

To exploit the leak we need to **assume (or prove)**, that the total number of positive pairs is small, compared to the total number of pairs. For example: think about an image dataset with $1000$ classes, $N$ images per class. Then if the task was to tell whether a pair of images belongs to the same class or not, we would have $1000\frac{N(N-1)}{2}$ positive pairs, while total number of pairs was $\frac{1000N(1000N - 1)}{2}$.

Another example: in [Quora competitition](https://www.kaggle.com/c/quora-question-pairs) the task was to classify whether a pair of qustions are duplicates of each other or not. Of course, total number of question pairs is very huge, while number of duplicates (positive pairs) is much much smaller.

Finally, let's get a fraction of pairs of class `1`. We just need to submit a constant prediction "all ones" and check the returned accuracy. Create a dataframe with columns `pairId` and `Prediction`, fill it and export it to `.csv` file. Then submit to grader and examine grader's output. 

So, we assumed the total number of pairs is much higher than the number of positive pairs, but it is not the case for the test set. It means that the test set is constructed not by sampling random pairs, but with a specific sampling algorithm. Pairs of class `1` are oversampled.

Now think, how we can exploit this fact? What is the leak here? If you get it now, you may try to get to the final answer yourself, othewise you can follow the instructions below.   

In [11]:
# YOUR CODE GOES HERE
df = pd.DataFrame()
df['pairId'] = test['pairId']
df['Prediction'] = 1
df.to_csv('submission.csv', index=False)
df.head()

# grader reports accuracy score of 0.5. this means that half of the test cases are 1 and half 0. 
# if we assume that the number of positive pairs in the training set is much less than 1, and combine
#     this with the fact that half of the test set pairs are 1, we can deduce that the test set is
#     not randomly generated. there are too many pairs of 1 in test
# of the 368,550 pairs, 184,275 will be classified as 1. 

,pairId,Prediction
0,0,1
1,1,1
2,2,1
3,3,1
4,4,1


# Building a magic feature

In this section we will build a magic feature, that will solve the problem almost perfectly. The instructions will lead you to the correct solution, but please, try to explain the purpose of the steps we do to yourself -- it is very important.

## Incidence matrix

First, we need to build an [incidence matrix](https://en.wikipedia.org/wiki/Incidence_matrix). You can think of pairs `(FirstId, SecondId)` as of edges in an undirected graph. 

The incidence matrix is a matrix of size `(maxId + 1, maxId + 1)`, where each row (column) `i` corresponds `i-th` `Id`. In this matrix we put the value `1` to the position `[i, j]`, if and only if a pair `(i, j)` or `(j, i)` is present in  a given set of pais `(FirstId, SecondId)`. All the other elements in the incidence matrix are zeros.   

**Important!** The incidence matrices are typically very very sparse (small number of non-zero values). At the same time incidence matrices are usually huge in terms of total number of elements, and it is **impossible to store them in memory in dense format**. But due to their sparsity incidence matrices **can be easily represented as sparse matrices**. If you are not familiar with sparse matrices, please see [wiki](https://en.wikipedia.org/wiki/Sparse_matrix) and [scipy.sparse reference](https://docs.scipy.org/doc/scipy/reference/sparse.html). Please, use any of `scipy.sparse` constructors to build incidence matrix. 

For example, you can use this constructor: `scipy.sparse.coo_matrix((data, (i, j)))`. We highly recommend to learn to use different `scipy.sparse` constuctors, and matrices types, but if you feel you don't want to use them, you can always build this matrix with a simple `for` loop. You will need first to create a matrix using `scipy.sparse.coo_matrix((M, N), [dtype])` with an appropriate shape `(M, N)` and then iterate through `(FirstId, SecondId)` pairs and fill corresponding elements in matrix with ones. 

**Note**, that the matrix should be symmetric and consist only of zeros and ones. It is a way to check yourself.

In [58]:
#inc_mat = # YOUR CODE GOES HERE (but probably you will need to write few more lines before)
#i = test['FirstId'].max() + 1
#j = test['SecondId'].max() + 1
#scipy.sparse.coo_matrix((data, (i, j))

# need to create a sparse matrix of First/Second Ids serving as both row and column indices. 
# we'll create two lists and concatenate them. one is for forward edges (FirstId -> SecondId)
#   and vice versa. we'll concatenate the lists and then remove dupicates. 
# create forward edges and backward edges; concatenate; remove duplicate entries (there are 228)
edges_fw = test[['FirstId', 'SecondId']].values
edges_bw = test[['SecondId', 'FirstId']].values
edges = np.concatenate([edges_fw, edges_bw])

# somehow this operation results in 228 duplicated rows
s = pd.DataFrame(edges)
print(s[s.duplicated()].shape)

edges = np.unique(edges, axis=0)
#print(edges.shape)

# create sparse matrix
# create a vector of ones corresponding to the number of unique edges
data = np.ones(len(edges))

# determine row and column indices
i = row_indices = edges[:,0]      # first column of Ids   FirstId : SecondId
j = column_indices = edges[:, 1]  # second column of Ids  SecondId : FirstId

# ones are placed where (FirstId, SecondId) & vice versa correspond to a pair in test
inc_mat = scipy.sparse.coo_matrix((data, (i, j)))
'''
  coo_matrix((data, (i, j)), [shape=(M, N)])
    to construct from three arrays:
            data[:] the entries of the matrix, in any order
            i[:] the row indices of the matrix entries
            j[:] the column indices of the matrix entries
    Where A[i[k], j[k]] = data[k]. When shape is not specified, it is inferred from the index arrays
'''

# Sanity checks
assert inc_mat.max() == 1
assert inc_mat.sum() == 736872

(228, 2)


In [59]:
print(inc_mat.shape)

# there are 26,325 unique First/Second Ids and 736,872 edges (or, unique relationships between FirstId and SecondId)
# the number of 1's in the matrix is equal to the sum over the matrix

(26325, 26325)


In [14]:
# inc_mat is a matrix with FirstIds and SecondIds serving as both the row and column indices. 
# the matrix is marked with zeros except where there is a relationship, or an edge,
#   between a FirstId and a SecondId or vice versa

# the matrix is created as follows:
#   create a matrix m of zeros of size row, column
#   create a vector v of ones of size row
#   iterate simultaneously through row and column
#      maintain the sum of each row, column pair by adding the corresponding value from data
#   collapse the row, column sums into matrix m

It is convenient to have matrix in `csr` format eventually.

In [15]:
inc_mat = inc_mat.tocsr()

## Now build the magic feature

Why did we build the incidence matrix? We can think of the rows in this matix as of representations for the objects. `i-th` row is a representation for an object with `Id = i`. Then, to measure similarity between two objects we can measure similarity between their representations. And we will see, that such representations are very good.

Now select the rows from the incidence matrix, that correspond to `test.FirstId`'s, and `test.SecondId`'s.

In [16]:
# Note, scipy goes crazy if a matrix is indexed with pandas' series. 
# So do not forget to convert `pd.series` to `np.array`
# These lines should normally run very quickly 
rows_FirstId   = inc_mat[np.array(test['FirstId'])] # YOUR CODE GOES HERE
rows_SecondId  = inc_mat[np.array(test['SecondId'])] # YOUR CODE GOES HERE

print(rows_FirstId.shape)
print(rows_SecondId.shape)

# This code retrieves from the incidence matrix two sets of rows: one indexed by FirstId
#   and the other by SecondId. 
# Each is a sparse matrix. 
# Each row can be intepretted as telling us the edges between the row index and the
#   column indexes (which are all the unique First/Second Ids)
# Each row tells us the connectivity between a First/Second Id and all the others. 

# rows_First_Id tells us what FirstId connects to
# rows_Second_Id tells us what SecondId connects to

(368550, 26325)
(368550, 26325)


Our magic feature will be the *dot product* between representations of a pair of objects. Dot product can be regarded as similarity measure -- for our non-negative representations the dot product is close to 0 when the representations are different, and is huge, when representations are similar. 

Now compute dot product between corresponding rows in `rows_FirstId` and `rows_SecondId` matrices.

In [17]:
# Note, that in order to do pointwise multiplication in scipy.sparse you need to use function `multiply`
# regular `*` corresponds to matrix-matrix multiplication

#f = # YOUR CODE GOES HERE

# dot product has to be calculated 'manually' (element wise multiplication followed by summation)
f = rows_FirstId.multiply(rows_SecondId)
f = f.sum(axis=1)
f = np.squeeze(np.asarray(f)) # go from (368550, 1) to (268440, )

# Sanity check
assert f.shape == (368550, )

# what does the dot product tell us?
# it indicates the similiarity between what FirstIds connect to and what SecondIds connect to
# each Pair will now have a similiarity score

That is it! **We've built our magic feature.** 

# From magic feature to binary predictions

But how do we convert this feature into binary predictions? We do not have a train set to learn a model, but we have a piece of information about test set: the baseline accuracy score that you got, when submitting constant. And we also have a very strong considerations about the data generative process, so probably we will be fine even without a training set. 

We may try to choose a thresold, and set the predictions to 1, if the feature value `f` is higer than the threshold, and 0 otherwise. What threshold would you choose? 

How do we find a right threshold? Let's first examine this feature: print frequencies (or counts) of each value in the feature `f`.

In [18]:
# For example use `np.unique` function, check for flags

print(np.unique(f, return_counts=True))

# there are only 7 unique values of the dot product. Look at the return counts.
# we know that 50% of the test (184,275) items must be classified as 1 
# we can use the similiaity score as a proxy to split the test set into two parts
# 
# all return counts occur when dot product is greater than 14
# of the 368,550 pairs, 104,275 will be classified as 1
# we clearly can get that number by setting f > 15. 
# this will cause 184,419 of the pairs to be set to 1 and results in 
#    an accuracy score of 0.999609. 

(array([ 14.,  15.,  19.,  20.,  21.,  28.,  35.]), array([183279,    852,    546, 183799,      6,     54,     14]))


Do you see how this feature clusters the pairs? Maybe you can guess a good threshold by looking at the values? 

In fact, in other situations it can be not that obvious, but in general to pick a threshold you only need to remember the score of your baseline submission and use this information. Do you understand why and how?  

Choose a threshold below: 

In [19]:
pred = f > 15 # SET THRESHOLD HERE

# Finally, let's create a submission

In [20]:
submission = test.loc[:,['pairId']]
submission['Prediction'] = pred.astype(int)

submission.to_csv('submission_4.csv', index=False)

# with f > 15, score was 0.999609 
# with f > 14, score was 0.997298
# with f > 19, score was 0.998128
# best result is f > 15

Now submit it to the grader! It is not possible to submit directly from this notebook, as we need to submit a `csv` file, not a single number (limitation of Coursera platform). 

To download `submission.csv` file that you've just produced <a href='./submission.csv'>click here</a> (if the link opens in browser, right-click on it and shoose "Save link as"). Then go to [assignment page](https://www.coursera.org/learn/competitive-data-science/programming/KsASv/data-leakages/submission) and submit your `.csv` file in 'My submission' tab.


If you did everything right, the score should be very high.

**Finally:** try to explain to yourself, why the whole thing worked out. In fact, there is no magic in this feature, and the idea to use rows in the incidence matrix can be intuitively justified.

# Bonus

Interestingly, it is not the only leak in this dataset. There is another totally different way to get almost 100% accuracy. Try to find it!